In [15]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModel
from torch import nn

In [16]:
import numpy as np
import pandas as pd

In [17]:
transformers.__version__

'4.2.2'

# Drug-ADE raw file

## Download data

In [35]:
%%bash
mkdir -p data

cd data
# 25M
wget https://fis.fda.gov/content/Exports/faers_ascii_2013q1.zip
# 30M
wget https://fis.fda.gov/content/Exports/faers_ascii_2014q1.zip


for f in `ls -1 *.zip`; 
do unzip $f -d `basename $f .zip`; 
done

# rm faers_ascii_2013q1.zip
# rm faers_ascii_2014q1.zip

--2021-02-15 18:59:42--  https://fis.fda.gov/content/Exports/faers_ascii_2013q1.zip
Resolving fis.fda.gov (fis.fda.gov)... 150.148.132.93, 2607:f220:300:8008::5d
Connecting to fis.fda.gov (fis.fda.gov)|150.148.132.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘faers_ascii_2013q1.zip’

     0K .......... .......... .......... .......... ..........  304K
    50K .......... .......... .......... .......... ..........  397K
   100K .......... .......... .......... .......... .......... 1.30M
   150K .......... .......... .......... .......... ..........  617K
   200K .......... .......... .......... .......... ..........  614K
   250K .......... .......... .......... .......... ..........  611K
   300K .......... .......... .......... .......... ..........  634K
   350K .......... .......... .......... .......... .......... 14.3M
   400K .......... .......... .......... .......... ..........  625K
   450K .......... ..

## Training data

In [38]:
tmp_drug = pd.read_csv("data/faers_ascii_2013q1/ascii/DRUG13Q1.txt", sep="$")
tmp_reac = pd.read_csv("data/faers_ascii_2013q1/ascii/REAC13Q1.txt", sep="$")


/home/xiruod/anaconda3/envs/mytest/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (9,10,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# Testing: only use 1000 rows
tmp_drug = tmp_drug[:1000].copy()

In [20]:
tmp_dataset = tmp_drug.merge(tmp_reac, on="primaryid", how="inner")

tmp_dataset = tmp_dataset[["primaryid", 'drugname', 'role_cod', 'pt']].copy()

tmp_dataset['drugname'] = tmp_dataset['drugname'].str.lower()
tmp_dataset['pt'] = tmp_dataset['pt'].str.lower()

tmp_dataset.dropna(inplace=True)

In [21]:
tmp_dataset.shape

(43627, 4)

In [22]:
tmp_dataset[tmp_dataset.primaryid == 30375293]

,primaryid,drugname,role_cod,pt
0,30375293,avonex,PS,amenorrhoea
1,30375293,avonex,PS,asthenia
2,30375293,avonex,PS,bladder disorder
3,30375293,avonex,PS,blood pressure increased
4,30375293,avonex,PS,dehydration
...,...,...,...,...
115,30375293,imuran,C,renal disorder
116,30375293,imuran,C,renal failure
117,30375293,imuran,C,urinary incontinence
118,30375293,imuran,C,urinary retention


Assumption #1: for each `primaryid`, there is no difference in `pt`, i.e., if one drug is `PS` for one `pt`, it is also `PS` for all other `pt`s; same as `SS` and `C`

In [23]:
tmp_dataset[tmp_dataset.primaryid == 30375293].groupby("drugname")['pt'].nunique()

drugname
amantadine       15
avonex           15
imuran           15
naprelan         15
steroid (nos)    15
zanaflex         15
zoloft           15
Name: pt, dtype: int64

In [24]:
ret_dict = {"primaryid":[], 'pt':[], 'drug':[], 'role_cod':[]}

def collect_all(df, ret, rand_seed):
    # df is already grouped by 'primaryid'
    # to be used with DataFrame.apply()
    
    ret['primaryid'].append(df.name)
    
    ret['pt'].append(df['pt'].unique().tolist())
    
    tmp = df[['drugname','role_cod']].copy()
    
    # NOTE: assumption made: for one drug, there is ONLY ONE role_cod
    # NOTE update: the above assumption is not true... most of times..
    #              so, for drugname with multiple roles, pick the first one in "drug_seq"
    #              This method is heavily depending on the fact that raw data is ordered by drug_seq!
    # tmp = tmp[~tmp[['drugname']].duplicated(keep="first")]
    
    tmp.drop_duplicates(['drugname'], keep='first', inplace=True)
    
    # shuffle tmp -> in case PS always comes first..
    tmp = tmp.sample(frac=1, random_state=rand_seed)
    
    ret['drug'].append(tmp['drugname'].tolist())
    ret['role_cod'].append(tmp['role_cod'].tolist())

    return None

In [25]:
param_seed = 2991
tmp_dataset.groupby("primaryid").apply(collect_all, ret=ret_dict, rand_seed=param_seed)

""


In [26]:
i = 5
for k,v in ret_dict.items():
    print(k, ": ", v[i])

primaryid :  37108102
pt :  ['arthralgia', 'disturbance in attention', 'dizziness', 'hepatic function abnormal', 'muscle spasms', 'myalgia']
drug :  ['lopressor', 'salospir-a', 'hovalin']
role_cod :  ['C', 'C', 'PS']


# Pre-process through PubMedBERT

In [27]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")  
pubmed_bert = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [29]:
# max length of one WordPiece-tokenized report (for truncating and padding)
# This may not be long enough for some reports
param_max_length = 100

In [30]:
# encoding
all_encodings = tokenizer(ret_dict['drug'], ret_dict['pt'], max_length=param_max_length, padding="max_length", truncation=True, 
                is_split_into_words=True, return_offsets_mapping=True)

In [31]:
# Drug Rold Codes
unique_roles = set(["PS", "SS", "C", "I"])

# hard code role_cod to numeric categories
role_2_id = {"PS":0, "SS":1, "C":2, "I":3}
id_2_role = {value:key for key, value in role_2_id.items()}


n_roles = len(unique_roles)

In [32]:
print(role_2_id)
print(id_2_role)

{'PS': 0, 'SS': 1, 'C': 2, 'I': 3}
{0: 'PS', 1: 'SS', 2: 'C', 3: 'I'}


In [33]:
encoded_labels = []
for report_labels, report_offset in zip(ret_dict['role_cod'], all_encodings['offset_mapping']):
    report_labels_num = [role_2_id[_] for _ in report_labels]
    # NOTE: -999 is arbitrary here...
    report_long_labels = np.ones(len(report_offset),dtype=int) * -999
    
    arr_offset = np.array(report_offset)
    
    # positions of special character [CLS] drug [SEP] ADE [SEP]
    tmp = np.where((arr_offset[:,0] == 0) & (arr_offset[:,1] == 0))
    
    # find positions of labels whose first offset position is 0 and the second is not 0
    # in drug part. Set those in ADE part to false, i.e., keep them -999 -> no use for predictions
    is_start = ((arr_offset[:,0] == 0) & (arr_offset[:,1] != 0))
    is_start[tmp[0][1]:] = False
    
    # sum(is_start): in case of truncation, where drug got truncated, if so, also truncate labels
    report_long_labels[is_start] = report_labels_num[:sum(is_start)]
    encoded_labels.append(report_long_labels.tolist())
    
    

In [34]:
# This could be casted to all samples
# use position 1
tmp_pos = torch.tensor([1] * param_max_length).type(torch.LongTensor)

# Model!

## Params

In [27]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [28]:
param_fine_tune_hidden_size = 512

#define a batch size
batch_size = 32 # 32

param_device = torch.device("cuda")

## Freeze PubMedBERT parameters

In [29]:
for param in pubmed_bert.parameters():
    param.requires_grad = False

## DataLoader

In [30]:

# wrap tensors
train_data = TensorDataset(torch.tensor(all_encodings['input_ids']), torch.tensor(all_encodings['attention_mask']),
                           torch.tensor(all_encodings['token_type_ids']), torch.tensor(encoded_labels))

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


## Main Model

In [31]:
class Fine_Tune_Multilevel(nn.Module):

    def __init__(self, bert, hidden_size, category_size):
      
        super(Fine_Tune_Multilevel, self).__init__()
        
        self.bert = bert 
      
        param_bert_size = pubmed_bert.config.hidden_size
        
        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(param_bert_size, hidden_size) #(768, 512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(hidden_size, category_size)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=2)

    #define the forward pass
    def forward(self, sent_id, attention_mask, token_type_ids, position_ids):

        #pass the inputs to the model
        ## NOTE: this unpacking may not work...
        ret_bert = self.bert(sent_id, attention_mask=attention_mask, 
                              token_type_ids=token_type_ids, position_ids=position_ids)

        x = self.fc1(ret_bert[0])

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)

        return x

In [32]:
model_fine_tune = Fine_Tune_Multilevel(pubmed_bert, param_fine_tune_hidden_size, n_roles)
model_fine_tune = model_fine_tune.to(param_device)

In [33]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model_fine_tune.parameters(),
                  lr = 1e-5)          # learning rate

In [34]:
# define the loss function
# cross_entropy  = nn.NLLLoss() 
cross_entropy = nn.NLLLoss()

# number of training epochs
epochs = 10

In [35]:
def train(model, device, train_dataloader, positions, n_categories):
  
    model.train()

    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    
    positions = positions.to(device)
    
    # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        
        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        model_ret = model(sent_id=batch[0], attention_mask=batch[1],
                           token_type_ids=batch[2], position_ids=positions)

        # calculate loss: remove labels with -999
        preds = model_ret.view(-1, n_categories)
        target = batch[3].flatten()

        loss = cross_entropy(preds[target != -999], target[target != -999])


        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push itk to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

### Start Training!

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
# valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train(model=model_fine_tune, device=param_device, train_dataloader=train_dataloader,
                             positions=tmp_pos, n_categories=n_roles)
    
    #evaluate model
    # valid_loss, _ = evaluate()
    
    
    
    # append training and validation loss
    train_losses.append(train_loss)
    # valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    # print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10


For future use of loooooong run: store output(print)

%%capture stored_output   
stored_output.show()

### Check on training performances

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.pointplot(x=np.arange(len(train_losses)),y=train_losses)
ax.set(xlabel='Training Epochs', ylabel='Training Error')

In [ ]:
train_losses

Check on training performances

In [ ]:
tmp_model = model_fine_tune

In [ ]:
tmp_model = tmp_model.to("cpu")

In [ ]:
tmp_model = tmp_model.eval()

In [ ]:
i = 10
tmp_ret = tmp_model(torch.tensor(all_encodings['input_ids'])[:i], torch.tensor(all_encodings['attention_mask'])[:i],
                           torch.tensor(all_encodings['token_type_ids'])[:i], tmp_pos)

In [ ]:
for preds, labels, primaryid, enc_drugs, pt, raw_drugs, offset_mappings in zip(tmp_ret.argmax(dim=2), torch.tensor(encoded_labels)[:i], ret_dict['primaryid'][:i], all_encodings['input_ids'][:i], ret_dict['pt'][:i],ret_dict['drug'][:i], all_encodings['offset_mapping'][:i]):
    print(primaryid, " pt list: ", pt)
    
    token_drugs = tokenizer.convert_ids_to_tokens(enc_drugs)
    print("drug list: ", raw_drugs)
    counter_sep = 0
    for _pred,_label,_drug, _offset in zip(preds, labels, token_drugs, offset_mappings):
        if _drug == "[SEP]":
            counter_sep += 1
            
        txt_prt = "{:>12} {:<10} {:<5d} {:<5d}".format(primaryid, _drug, _pred, _label)
        
        if (_offset[0] == 0) and (_offset[1] != 0) and (counter_sep < 1):
            print('\033[91m', txt_prt)
        else:
            print('\033[90m',txt_prt)
        
        if counter_sep == 2:
            break


In [ ]:
print(id_2_role)

# Eval

## Evaluation Data Processing

In [63]:
tmp_drug = pd.read_csv("data/faers_ascii_2014q1/ascii/DRUG14Q1.txt", sep="$")
tmp_reac = pd.read_csv("data/faers_ascii_2014q1/ascii/REAC14Q1.txt", sep="$")


/home/xiruod/anaconda3/envs/mytest/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
tmp_dataset_eval = tmp_drug.merge(tmp_reac, on="primaryid", how="inner")

tmp_dataset_eval = tmp_dataset_eval[["primaryid", 'drugname', 'role_cod', 'pt']].copy()

tmp_dataset_eval['drugname'] = tmp_dataset_eval['drugname'].str.lower()
tmp_dataset_eval['pt'] = tmp_dataset_eval['pt'].str.lower()

tmp_dataset_eval.dropna(inplace=True)

In [ ]:
# Only select top 100,000 rows for evaluation here
tmp_dataset_eval = tmp_dataset_eval[:100000].copy()

In [65]:
ret_dict_eval = {"primaryid":[], 'pt':[], 'drug':[], 'role_cod':[]}

In [66]:
tmp_dataset_eval.groupby("primaryid").apply(collect_all, ret=ret_dict_eval, rand_seed=param_seed)

""


In [67]:
eval_encodings = tokenizer(ret_dict_eval['drug'], ret_dict_eval['pt'], max_length=param_max_length, padding="max_length", truncation=True, 
                is_split_into_words=True, return_offsets_mapping=True)

In [68]:
encoded_labels_eval = []
for report_labels, report_offset in zip(ret_dict_eval['role_cod'], eval_encodings['offset_mapping']):
    report_labels_num = [role_2_id[_] for _ in report_labels]
    # NOTE: -999 is arbitrary here...
    report_long_labels = np.ones(len(report_offset),dtype=int) * -999
    
    arr_offset = np.array(report_offset)
    
    # positions of special character [CLS] drug [SEP] ADE [SEP]
    tmp = np.where((arr_offset[:,0] == 0) & (arr_offset[:,1] == 0))
    
    # find positions of labels whose first offset position is 0 and the second is not 0
    # in drug part. Set those in ADE part to false, i.e., keep them -999 -> no use for predictions
    is_start = ((arr_offset[:,0] == 0) & (arr_offset[:,1] != 0))
    is_start[tmp[0][1]:] = False
    
    # sum(is_start): in case of truncation, where drug got truncated, if so, also truncate labels
    report_long_labels[is_start] = report_labels_num[:sum(is_start)]
    encoded_labels_eval.append(report_long_labels.tolist())
    
    

## Apply model

In [ ]:
# test on one report..
ii = 0
i =21
with torch.no_grad():
    eval_ret = tmp_model(torch.tensor(eval_encodings['input_ids'])[ii:i], torch.tensor(eval_encodings['attention_mask'])[ii:i],
                           torch.tensor(eval_encodings['token_type_ids'])[ii:i], tmp_pos)
 

In [69]:
# Apply on all evaluation set
# wrap tensors
eval_data = TensorDataset(torch.tensor(eval_encodings['input_ids']), torch.tensor(eval_encodings['attention_mask']),
                           torch.tensor(eval_encodings['token_type_ids']), torch.tensor(encoded_labels_eval))


# dataLoader for train set
eval_dataloader = DataLoader(eval_data, batch_size=batch_size)


In [ ]:
# NOTE: running this on CPU takes tooo long (~30 min). Should better move to GPU

eval_ret = []
for step, batch in enumerate(eval_dataloader):
    tmp_ret = tmp_model(sent_id=batch[0], attention_mask=batch[1],
                           token_type_ids=batch[2], position_ids=tmp_pos)
    
    eval_ret.extend(tmp_ret.tolist())

eval_ret = torch.tensor(eval_ret)

In [ ]:
eval_ret.size()

In [ ]:
torch.tensor(encoded_labels_eval).size()

In [ ]:
tmp_ret.size()

In [ ]:
preds = eval_ret.view(-1, n_roles)
target = torch.tensor(encoded_labels_eval).flatten()

loss = cross_entropy(preds[target != -999], target[target != -999])


In [ ]:
loss

In [ ]:
ax = sns.pointplot(x=np.arange(len(train_losses)),y=train_losses)
# ax.axhline(loss.item(), ls='--', c="red")
ax.scatter(9, loss.item(), marker="o",c="red")
ax.set(xlabel='Training Epochs', ylabel='Error')

In [ ]:
counter_prt = 0

for preds, labels, primaryid, enc_drugs, pt, raw_drugs, offset_mappings in zip(eval_ret.argmax(dim=2), torch.tensor(encoded_labels_eval)[:i], ret_dict_eval['primaryid'][:i], eval_encodings['input_ids'][:i], ret_dict_eval['pt'][:i],ret_dict_eval['drug'][:i], eval_encodings['offset_mapping'][:i]):
    print(primaryid, " pt list: ", pt)
    
    token_drugs = tokenizer.convert_ids_to_tokens(enc_drugs)
    print("drug list: ", raw_drugs)
    counter_sep = 0
    for _pred,_label,_drug, _offset in zip(preds, labels, token_drugs, offset_mappings):
        if _drug == "[SEP]":
            counter_sep += 1
            
        txt_prt = "{:>12} {:<10} {:<5d} {:<5d}".format(primaryid, _drug, _pred, _label)
        
        if (_offset[0] == 0) and (_offset[1] != 0) and (counter_sep < 1):
            print('\033[91m', txt_prt)
        else:
            print('\033[90m',txt_prt)
        
        if counter_sep == 2:
            break
        
    counter_prt += 1
    if counter_prt == 10:
        break

In [ ]:
preds_ls = []
target_ls = []
for preds, labels, primaryid, enc_drugs, pt, raw_drugs, offset_mappings in zip(eval_ret.argmax(dim=2), torch.tensor(encoded_labels_eval), ret_dict_eval['primaryid'], eval_encodings['input_ids'], ret_dict_eval['pt'],ret_dict_eval['drug'], eval_encodings['offset_mapping']):
    
    
    token_drugs = tokenizer.convert_ids_to_tokens(enc_drugs)

    counter_sep = 0
    for _pred,_label,_drug, _offset in zip(preds, labels, token_drugs, offset_mappings):
        if _drug == "[SEP]":
            counter_sep += 1
            
        
        
        if (_offset[0] == 0) and (_offset[1] != 0) and (counter_sep < 1):
            preds_ls.append(_pred.item())
            target_ls.append(_label.item())
        
        
        if counter_sep == 2:
            break
        


In [ ]:
eval_df = pd.DataFrame(zip(preds_ls, target_ls), columns=['pred', 'target'])

In [ ]:
eval_df.groupby(['pred','target']).size().to_frame(name = 'size').reset_index()

In [ ]:
tmp = eval_df.groupby(['pred','target']).size().to_frame(name = 'size').reset_index()
pd.concat([tmp, pd.Series(["{:.2f}%".format(i) for i in tmp.groupby('pred')['size'].apply(lambda x: 100*x/x.sum())]).rename("Percentage")], axis=1)


In [ ]:

tmp = eval_df.groupby(['target', 'pred']).size().to_frame(name = 'size').reset_index()
pd.concat([tmp, pd.Series(["{:.2f}%".format(i) for i in tmp.groupby('target')['size'].apply(lambda x: 100*x/x.sum())]).rename("Percentage")], axis=1)
